### Status
- no existing library fulfills all requirements 
- core problem: model firmly baked into code everywhere in all of them
- no integrated runable system at the moment
- no user interface
- only able to analyze files
- but all necessary parts
  - abstractions for recording, preprocessor, model 
  - model, preprocessing disentangled from rest => allows runtime replacement 
  
- `birdnetlib` provides at the very least starting points for continously running service via `arecord` on linux and for the missint nuts and bolts 

### Code example

In [ ]:
import iSparrow.sparrow_model_base as spm 
import iSparrow.sparrow_recording as spr 
import iSparrow.preprocessor_base as spb
import iSparrow.utils as utils

import install_sparrow as sp

In [ ]:
from pathlib import Path

In [ ]:
# make a mock install of sparrow 
sp.install()

In [ ]:
# variables for analysis 
model_path= "birdnet_default"
sigmoid_sensitivity= 1.0
num_threads= 12
min_conf= 0.25

# variables for recording
recording_path: sp.EXAMPLES / "soundscape.wav"
sample_rate: 48000
overlap: 0.0
sample_secs: 3.0
resample_type: "kaiser_fast"

#### steps
- read module defined as input 
- build preprocessor
- build model 
- build recording instance from them 
- analyze

In [ ]:
ppd = utils.load_module("ppm", sp.MODELS/Path("birdnet_default") / "preprocessor.py")